In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import torch

from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi import analysis as analysis

from Simulation import Simulation
from CCM import CCM 

# Single population 

## With bistability constraint 

### q free

In [4]:
# PARAMETERS & FUNCTIONS SET-UP #
#################################

computer = "cpu"
enc = torch.float64
dev = torch.device(computer)

Ae, Ap, As, Av = 169, 268, 709, 634
dof = torch.as_tensor([
    
    Ae, Ap, As, Av, # Ae, Ap, As, Av (4)
    # wee, 	  wpe,     wse,     wes,     wvs,     wep,     wpp,     wsp,     wev,     wsv (10) :
    .136*Ae, .101*Ap, .002*As, .077*Ae, .048*Av, .112*Ae, .093*Ap, .0*As, .041*Ae, .001*As, 
    3.9, 4.5, 3.6, 2.9, # Ie_ext, Ip_ext, Is_ext, Iv_ext (5)
    0.8, .0598*Ae, .01 # q, J_adp, sigma (3)
    
], device=dev, dtype=enc)


dim = 21 # Number of degrees of freedom of the simulator [A_x (4), w_xy (10), I_ext-x (4), q, J_adp, sigma]

# PRIORS
mA, mW, mI, mP = [1. for k in range(4)], [.1 for k in range(10)], [.1 for k in range(4)], [0, .1, .001]
MA, MW, MI, MP = [1000. for k in range(4)], [75. for k in range(10)], [5. for k in range(4)], [1, 15., .02] 
priors_min, priors_max = np.concatenate((mA, mW, mI, mP)), np.concatenate((MA, MW, MI, MP))
priors = np.array([priors_min, priors_max])

prior_min = torch.as_tensor(priors_min, device=dev, dtype=torch.float64)
prior_max = torch.as_tensor(priors_max, device=dev, dtype=torch.float64)
prior = utils.BoxUniform(low=prior_min, high=prior_max)

# Model & simulation parameters, empirical data
mod_prm = torch.as_tensor([.020, .600, 4.5, 0., 1.9, 2.6, 1.5, 1.2, 7., 7., 7., 7., 1/45, 0.005], device=dev, dtype=enc)
sim_prm = torch.as_tensor([1000, 1., 1e-12, 1e-3, np.nan], device=dev, dtype=enc)
x_o = torch.as_tensor([27.5, 5.62, 5.6, 21.5, 1.75, 1.04, 1.12, 1.33, 60.2, 35.3, 35.2, 68.8, 21.7, 4.2], device=dev, dtype=torch.float64)

# SIMULATOR 
def simulator(theta):
    ccm = CCM(theta, mod_prm)
    sim = ccm.simulate(sim_prm)
    return sim.postproc()

simulator, prior = prepare_for_sbi(simulator, prior) # So that we are sure they comply with sbi

inference = SNPE(prior=prior)

In [5]:
# INFERENCE #
#############

num_rounds = 2

posteriors = []
proposal = prior

for _ in range(num_rounds):
    theta, x = simulate_for_sbi(simulator, proposal, num_simulations=10000)
    density_estimator = inference.append_simulations(theta, x, proposal=proposal).train()
    posterior = inference.build_posterior(density_estimator)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(x_o)

Running 10000 simulations.:   0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

### q fixed

### No amplitudes

## Without bistability constraint

# Full model multi-round SNPE-C with raw traces

# Reduced models